citation

In [1]:
import numpy as np 
import pandas as pd
from pathlib import Path
import os, shutil, time, csv,math
from os import walk
import cv2

## Variables

In [2]:
#Directory where all thre training data is stored
Top_dir = r'D:\Training_Data_Creation'
#lists of sites and image types
megatile_size=[40,40]
Tilesizes=[3,5,7]
imgtypes=['123','456','789','m']

## Defining Initial Functions

In [3]:
def process_tiles_mega(foldtype):
    '''This is for datasets which will not be compared and we want all the pixel values in the csv'''
    B1,B2,B3,B4,B5,B6,B7,B8,B9,BM=[],[],[],[],[],[],[],[],[],[]
    for imgtype in imgtypes: 
        #defining vars for different sources
        read_dir=Top_dir+'\\{}_Mega_{}\\Megatiles_{}\\'.format(foldtype,megatile_size[0],imgtype)
        img_size=megatile_size[0]
        for (dirpath, dirnames, filenames) in walk(read_dir):
            for file in filenames:
                if imgtype == '123':
                    #read image as 3 for raw 16bit
                    img= cv2.imread(read_dir+file,3)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    for i in range(img_size):
                        for j in range(img_size):
                            B1.append(str(abs(img[i,j][0])))#append the pixels to the respective lists
                            B2.append(str(abs(img[i,j][1])))
                            B3.append(str(abs(img[i,j][2])))
                if imgtype == '456':
                    img= cv2.imread(read_dir+file,3)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    for i in range(img_size):
                        for j in range(img_size):
                            B4.append(str(abs(img[i,j][0])))
                            B5.append(str(abs(img[i,j][1])))
                            B6.append(str(abs(img[i,j][2])))
                if imgtype == '789':
                    img= cv2.imread(read_dir+file,3)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    for i in range(img_size):
                        for j in range(img_size):
                            B7.append(str(abs(img[i,j][0])))
                            B8.append(str(abs(img[i,j][1])))
                            B9.append(str(abs(img[i,j][2])))
                if imgtype == 'm':
                    img= cv2.imread(read_dir+file,3)
                    for i in range(img_size):
                        for j in range(img_size):
                            BM.append(str(abs(img[i,j][0])))
    return B1,B2,B3,B4,B5,B6,B7,B8,B9,BM

In [4]:
def process_tiles_TS(foldtype,Tilesize):
    '''This is for datasets where we only care about the middle pixel of the largest tile size.
    The reason is because this will allow us to compare across tilesizes and models'''
    B1,B2,B3,B4,B5,B6,B7,B8,B9,BM=[],[],[],[],[],[],[],[],[],[]
    for imgtype in imgtypes: 
        #defining vars for different sources
        read_dir=Top_dir+'\\{}_Tiles_ts{}-mega{}\\Tiles_{}\\'.format(foldtype,Tilesize,
                                                                                   megatile_size[0],imgtype)
        img_size=megatile_size[0]
        for (dirpath, dirnames, filenames) in walk(read_dir):
            for file in filenames:
                if imgtype == '123':
                    #read image as 3 for raw 16bit
                    img= cv2.imread(read_dir+file,3)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    mid_pix=img[int(Tilesize/2-.5),int(Tilesize/2-.5)]#find middle pixel
                    B1.append(str(abs(mid_pix[0]))) #append the middle pixels to the respective lists
                    B2.append(str(abs(mid_pix[1])))
                    B3.append(str(abs(mid_pix[2])))
                if imgtype == '456':
                    img= cv2.imread(read_dir+file,3)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    mid_pix=img[int(Tilesize/2-.5),int(Tilesize/2-.5)]
                    B4.append(str(abs(mid_pix[0])))
                    B5.append(str(abs(mid_pix[1])))
                    B6.append(str(abs(mid_pix[2])))
                if imgtype == '789':
                    img= cv2.imread(read_dir+file,3)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    mid_pix=img[int(Tilesize/2-.5),int(Tilesize/2-.5)]
                    B7.append(str(abs(mid_pix[0])))
                    B8.append(str(abs(mid_pix[1])))
                    B9.append(str(abs(mid_pix[2])))
                if imgtype == 'm':
                    img= cv2.imread(read_dir+file,3)
                    mid_pix=img[int(Tilesize/2-.5),int(Tilesize/2-.5)]
                    BM.append(str(abs(mid_pix[0])))
    return B1,B2,B3,B4,B5,B6,B7,B8,B9,BM

In [5]:
def create_csv(foldtype):
    tr=[BM,B1,B2,B3]
    tr2=[B4,B5,B6,B7,B8,B9]
    #create 2 dfs since 1 sometimes breaks a limit, then you add them together
    df = pd.DataFrame(list(zip(*tr))).add_prefix('Col').astype('int64')
    df2=pd.DataFrame(list(zip(*tr2))).add_prefix('Col').astype('int64')
    df_n=pd.concat([df, df2], axis=1)
    #naming the colums
    df_n.columns = ["Trueval",'B1','B2','B3','B4','B5','B6','B7','B8','B9']
    #drop all no data pixels, which can be determined by Trueval == 255
    df_n = df_n.drop(df_n[df_n.Trueval == 255].index)
    df_n.to_csv(r'D:\Training_Data_Creation\Pointer_files\{}_Tiles{}.csv'.format(foldtype,megatile_size[0]), index=False)
    print('completed creating {} csv'.format(foldtype))

## Process Data

In [6]:
for ft in ['Train','Val']:
    B1,B2,B3,B4,B5,B6,B7,B8,B9,BM=process_tiles_mega(ft)
    create_csv(ft)
#creating csv with middle pixel of the largest test  size for comparison
B1,B2,B3,B4,B5,B6,B7,B8,B9,BM=process_tiles_TS('Test',7)
create_csv('Test')

completed creating Train csv
completed creating Val csv
completed creating Test csv
